In [5]:
from pdpbox import pdp
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd


In [6]:
raw_data = pd.read_csv("../data/data_clean.csv")
data = raw_data[raw_data['birth date'] < 2020].copy()

data['age'] = data['parole board interview date'] - data['birth date']
data['jail duration'] = data['parole board interview date'] - data['year of entry']

others_parole_type =  ['PIE', 'SP CONSDR', 'ECPDO', 'MEDICAL','RESCISSION', 'DEPORT']
data['parole board interview type'] = data['parole board interview type'].replace(others_parole_type, 'OTHERS').replace('SUPP MERIT', 'MERIT TIME').replace('PV REAPP', 'REAPPEAR')

data = data.dropna(axis=0, subset=['crime 1 - class', 'parole eligibility date'])

df_one_hot = pd.get_dummies(data, columns=[
    "sex", "race / ethnicity"], drop_first=True)

df_one_hot = pd.get_dummies(df_one_hot, columns=[
    "crime 1 - class", "crime 2 - class",
    "crime 3 - class", "crime 4 - class",
    "parole board interview type"])

df_one_hot.drop(columns=['release date','birth date', 'year of entry'],inplace=True)

X_train, X_test, y_train, y_test = train_test_split(df_one_hot.drop('y',axis=1), df_one_hot['y'], 
                                                    stratify=df_one_hot['y'], test_size=0.3, random_state=42)

In [9]:
# Load the model from the .pkl file
xgb_classifier = pickle.load(open("../model.pkl", "rb"))


In [8]:
# Load the model from the .pkl file
xgb_classifier = pickle.load(open("../model.pkl", "rb"))

# Assume you are interested in the feature at index 5
feature_name = xgb_classifier.feature_names[5]

# Create the partial dependence plot
pdp_iso = pdp.pdp_isolate(
    model=xgb_classifier, dataset=pd.DataFrame(X_train, columns=xgb_classifier.feature_names), 
    model_features=xgb_classifier.feature_names, feature=feature_name)

# plot it
fig, axes = pdp.pdp_plot(pdp_iso, feature_name, plot_lines=True, frac_to_plot=100)

AttributeError: 'XGBClassifier' object has no attribute 'feature_names'